### Austin's ACM SIG KDD Demo: Spark Streaming using Weather data
##### Presented on 23-Aug-2017
##### Ulind Narang

Date: 21-July-2017 <br>
Description: This code reads a csv file and for every row in csv file writes a new json file

In [3]:
%python
# Get ACCESS_KEY and SECRET_KEY from IAM in AWS. Insert the keys into the quotes below.
# Create an S3 bucket and put the name (S3 bucket names are globally unique - so, just the bucket-name)
ACCESS_KEY = "YOUR_IAM_ACCESS_KEY"  
SECRET_KEY = "YOUR_IAM_SECRET_KEY"  
AWS_BUCKET_NAME = "YOUR_S3_BUCKET_NAME"
MOUNT_NAME = "YOUR_S3_BUCKET_LOCAL_MOUNTED_NAME"


dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

# You can add try and except to catch any errors while mounting the bucket
# Login to S3 console or AWS CLI for accessing the 


In [4]:
import json
import csv
inFile = '/dbfs/mnt/$MOUNT_NAME/travis_rawData.csv' 
outFilePath = '/dbfs/mnt/$MOUNT_NAME/json-files/'
fileReader = csv.DictReader(open(inFile))

In [5]:
def to_float(s):
    try:
      s = s.strip()
    except:
      s = ''
    
    try:
      fT = float(s)
    except:
      fT = 0.0
      
    # return float(s) if s else 0.0
    return fT

In [6]:
#import re
# for every row in input file, populate the dictionary row and write to a new file

import time
import datetime
    
for row in fileReader:
    dictRow = {}
    for column, value in row.items():
        dictRow.update({column: value})
    
    keysRequired = ['STATION', 'DATE', 'HOURLYDRYBULBTEMPF', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYStationPressure','HOURLYSeaLevelPressure','HOURLYPrecip']
    dictRowSelected = {x:dictRow[x] for x in keysRequired}
    
    tempDict = {}
    
    tempStn = dictRowSelected['STATION']
    tStnDict = {'STATION': tempStn}
    tempDict.update(tStnDict)
    
    tempDate = dictRowSelected['DATE']
    tempTimeStamp = time.mktime(datetime.datetime.strptime(tempDate, "%Y-%m-%d %H:%M").timetuple())
    tTSDict = {'UNIX_TIMESTAMP': tempTimeStamp}
    tempDict.update(tTSDict)
    
    keysFloatRequired = ['HOURLYDRYBULBTEMPF', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYStationPressure','HOURLYSeaLevelPressure','HOURLYPrecip']
    for key in keysFloatRequired:
      tempStr = dictRowSelected[key]
      tempFloat = to_float(tempStr)
      
      tStrDict = {}
      tStrDict = {key: tempFloat}
      tempDict.update(tStrDict)
    
  
    iTemp = tempDict['HOURLYDRYBULBTEMPF']
    tStr = ''
    
    if iTemp <=60:
      tStr = "0 - 60"
    elif iTemp > 60 and iTemp <=70:
      tStr = "60+ - 70"
    elif iTemp > 70 and iTemp <=80:
      tStr = "70+ - 80"
    elif iTemp > 80 and iTemp <=90:
      tStr = "80+ - 90"
    elif iTemp > 90 and iTemp <=100:
      tStr = "90+ - 100"
    else:
      tStr = "100+"
    
    tTempDict = {'TEMPRANGE': tStr}
    tempDict.update(tTempDict)
    
    print(tempDict)

  
    fileNumber = fileNumber + 1
    fileName = 'WBAN_13904_' + str(fileNumber) + '.json'
    outFile = outFilePath + fileName
    fileWriter = open(outFile, 'w')
        
  
    # no pretty print - Spark did not take pretty printed json files
    json.dump(tempDict, fileWriter)
        
    fileWriter.close()



In [7]:
# end of code